[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/21--no-sql-database-mongo/06--document-database_mongodb--data-analysis.ipynb)

In [2]:
!sudo apt install mongodb
!sudo service mongodb start

Unable to locate an executable at "/Library/Java/JavaVirtualMachines/adoptopenjdk-11.jdk/Contents/Home/bin/apt" (-1)
sudo: service: command not found


In [3]:
!mkdir -p var
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson -O var/video_movies.bson
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movieDetails.bson -O var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection movies var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection films var/video_movies.bson

--2021-11-20 11:36:22--  https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SouthbankSoftware/dbkoda-data/master/SampleCollections/dump/SampleCollections/video_movies.bson [following]
--2021-11-20 11:36:22--  https://raw.githubusercontent.com/SouthbankSoftware/dbkoda-data/master/SampleCollections/dump/SampleCollections/video_movies.bson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343355 (335K) [application/octet-stream]
Saving to: ‘var/video_movies.bson’

var/video_movies.bs 10

Aggregation framework

```json
{
     "_id" : ObjectId("5b107bec1d2952d0da904dd7"),
     "title" : "Titan A.E.",
     "year" : 2000,
     "rated" : "PG",
     "runtime" : 94,
     "countries" : [
             "USA"
     ],
     "genres" : [
             "Animation",
             "Action",
             "Adventure"
     ],
     "director" : "Don Bluth, Gary Goldman, Art Vitello",
     "writers" : [
             "Hans Bauer",
             "Randall McCormick",
             "Ben Edlund",
             "John August",
             "Joss Whedon"
     ],
     "actors" : [
             "Matt Damon",
             "Bill Pullman",
             "John Leguizamo",
             "Nathan Lane"
     ],
     "plot" : "A young man learns that he has to find a hidden Earth ship before an enemy alien species does in order to secure the survival of humanity.",
     "poster" : "http://ia.media-imdb.com/images/M/MV5BMjE0NTU0ODg4NV5BMl5BanBnXkFtZTcwNzY3MTQyMQ@@._V1_SX300.jpg",
     "imdb" : {
             "id" : "tt0120913",
             "rating" : 6.6,
             "votes" : 50875
     },
     "tomato" : {
             "meter" : 52,
             "image" : "rotten",
             "rating" : 5.7,
             "reviews" : 99,
             "fresh" : 51,
             "consensus" : "Great visuals, but the story feels like a cut-and-paste job of other sci-fi movies.",
             "userMeter" : 60,
             "userRating" : 3.2,
             "userReviews" : 69055
     },
     "metacritic" : 48,
     "awards" : {
             "wins" : 1,
             "nominations" : 7,
             "text" : "1 win & 7 nominations."
     },
     "type" : "movie"
}
```

In [1]:
#Connect to database
!pip install pymongo

You should consider upgrading via the '/Users/jkanclerz/myplace/dydaktyka/przetwarzanie-dokumnetow/data-science-workshops-2021/.venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://192.168.56.4:27017')

In [3]:
db = client.datascience

agregation operators
----------------------

* \$project -> schape


```json
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "imdb" : {
             "id" : "tt0120913",
             "rating" : 6.6,
             "votes" : 50875
     }
}
```

```json
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "rating" : 6.6
}
```

* \$match -> filtering
* \$group
* \$sort
* \$skip
* \$limit
* \$unwind

```json
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "actors" : [
     "Matt Damon",
     "Bill Pullman",
     "John Leguizamo"
 ],
}
```
unwind result in
```json
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "actors" : "Matt Damon"
},
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "actors" : "Bill Pullman"
},
{
 "title" : "Titan A.E.",
 "year" : 2000,
 "actors" : "John Leguizamo"
}
```


Which movie has the highest nomination to winning ratio?
----------------------------------------------------

In [4]:
pp(db.movies.find().next())

{'_id': ObjectId('569190cf24de1e0ce2dfcd74'),
 'actors': ['Cüneyt Arkin', 'Haldun Boysan', 'Berda Ceyhan', 'Veysel Diker'],
 'awards': {'nominations': 0, 'text': '', 'wins': 0},
 'countries': ['Turkey'],
 'director': 'Kartal Tibet',
 'genres': ['Action', 'Comedy', 'Sci-Fi'],
 'imdb': {'id': 'tt0808240', 'rating': 1.9, 'votes': 12638},
 'plot': 'A family of Turks try to adapt to life in a new solar system.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTYwOTMwODIxNF5BMl5BanBnXkFtZTcwMjc0NzA0MQ@@._V1_SX300.jpg',
 'rated': None,
 'released': datetime.datetime(2006, 12, 15, 5, 0),
 'runtime': 110,
 'title': 'Turks in Space',
 'type': 'movie',
 'writers': ['Murat Boyacioglu'],
 'year': 2006}


In [4]:
pp(list(db.movies.aggregate([
    {"$match": {"awards": {"$exists": "1"}}},
    {"$match": {"awards.wins": {"$gt": 0}, "awards.nominations": {"$gt": 0}}},
    {
        "$project": {
            "title": 1,
            "awards": 1,
            'rating': '$imdb.rating',
            "ratio": {
                "$divide": ['$awards.wins', '$awards.nominations']
            }
        }
    },
    {"$sort": {"ratio": -1}},
    {"$limit": 2}
])))

[{'_id': ObjectId('5692a53024de1e0ce2dfdca5'),
  'awards': {'nominations': 1, 'text': '22 wins & 1 nomination.', 'wins': 22},
  'rating': 7.6,
  'ratio': 22.0,
  'title': 'Au bout du monde'},
 {'_id': ObjectId('5692a47c24de1e0ce2dfdb63'),
  'awards': {'nominations': 1, 'text': '22 wins & 1 nomination.', 'wins': 22},
  'rating': 8.1,
  'ratio': 22.0,
  'title': 'Everything Will Be Ok'}]


Which actor has participated in the highest amount of movies?
-------------------------------------------------------------


In [7]:
#single movie
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors"}},
    {"$sort": {"rating": -1}},
    {"$limit": 5}
])))

[{'_id': ObjectId('5692a13b24de1e0ce2dfcea8'), 'actor': 'Marley Shelton'},
 {'_id': ObjectId('5692a13b24de1e0ce2dfcea9'), 'actor': 'Charlton Heston'},
 {'_id': ObjectId('5692a13b24de1e0ce2dfcea8'), 'actor': 'Freddy Rodríguez'},
 {'_id': ObjectId('5692a13b24de1e0ce2dfcea8'), 'actor': 'Rose McGowan'},
 {'_id': ObjectId('5692a13b24de1e0ce2dfcea8'), 'actor': 'Josh Brolin'}]


In [9]:
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors"}},
    {"$group": {"_id": "$actor", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 5}
])))

[{'_id': 'Tom Hanks', 'count': 8},
 {'_id': 'Natalie Portman', 'count': 8},
 {'_id': 'Louis C.K.', 'count': 8},
 {'_id': 'Scarlett Johansson', 'count': 7},
 {'_id': 'B.B. King', 'count': 7}]


Whoo hoo Tom hanks and Natalie Portman taking the lead

Which actor had highest average movie rating value?
----------------------------------------------------

In [10]:
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors", "rating": "$imdb.rating"}},
    {"$group": {"_id": "$actor", "avg_rating": {"$avg": "$rating"}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5}
])))

[{'_id': 'Nikita Devine', 'avg_rating': 9.6},
 {'_id': 'Tony DeSergio', 'avg_rating': 9.6},
 {'_id': 'Nichola Holt', 'avg_rating': 9.6},
 {'_id': 'Michelle Banks', 'avg_rating': 9.6},
 {'_id': 'Milan Baros', 'avg_rating': 9.5}]


hmm, are they one movie star?!

In [11]:
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors", "rating": "$imdb.rating"}},
    {"$group": {"_id": "$actor", "avg_rating": {"$avg": "$rating"}, "count": {"$sum": 1}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5}
])))

[{'_id': 'Nikita Devine', 'avg_rating': 9.6, 'count': 1},
 {'_id': 'Tony DeSergio', 'avg_rating': 9.6, 'count': 1},
 {'_id': 'Nichola Holt', 'avg_rating': 9.6, 'count': 1},
 {'_id': 'Michelle Banks', 'avg_rating': 9.6, 'count': 1},
 {'_id': 'Milan Baros', 'avg_rating': 9.5, 'count': 1}]


indeed, avg trap, lets check with at least 2 movies

In [18]:
pp(list(db.movies.aggregate([
    {"$match": {"actors": {"$exists": "1"}}},
    {"$unwind": "$actors"},
    {"$project": {"actor": "$actors", "rating": "$imdb.rating"}},
    {"$group": {
        "_id": "$actor",
        "avg_rating": {
            "$avg": "$rating"
        },
        "count": {
            "$sum": 1
        },
        "max_score": {
            "$max": "$rating"
        }
    }},
    {"$match": {"count": {"$gt": 2}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5}
])))

[{'_id': 'Mark Hamill',
  'avg_rating': 8.633333333333333,
  'count': 3,
  'max_score': 8.8},
 {'_id': 'Carrie Fisher',
  'avg_rating': 8.633333333333333,
  'count': 3,
  'max_score': 8.8},
 {'_id': 'Harrison Ford', 'avg_rating': 8.6, 'count': 4, 'max_score': 8.8},
 {'_id': 'Brian Johnson', 'avg_rating': 8.5, 'count': 6, 'max_score': 8.6},
 {'_id': 'AC/DC', 'avg_rating': 8.5, 'count': 3, 'max_score': 8.6}]


Hmm Harrison Ford !!! 

![](../images/ford.jpg)

Grouping operators
------------------
* \$first
* \$last
* \$max
* \$min
* \$avg